<a href="https://colab.research.google.com/github/ns-it/FL_Xray_Flower/blob/main/experiments_two_paths_unified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# الخلية 1: الاستيراد والتحضير
import yaml
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import flwr as fl
import numpy as np
# استدعاء دوالنا الخاصة من ملف utils.py
from utils import setup_kaggle_and_download, get_data_partitions, calculate_class_weights, save_experiment_results

# GPU Check
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)

# Load Config
def load_config():
    with open("config.yaml", 'r') as f:
        return yaml.safe_load(f)
CONF = load_config()

print(f"🚀 Experiment: {CONF['experiment_name']}")

In [ ]:
# الخلية 2: تحميل البيانات من Kaggle وتقسيمها
# 1. Download from Kaggle
data_path = setup_kaggle_and_download(
    CONF['kaggle']['dataset_handle'],
    CONF['kaggle']['download_path']
)

# 2. Partition Data
client_datasets, test_ds = get_data_partitions(
    data_dir=data_path,
    alpha=CONF['data']['alpha'],
    num_clients=CONF['data']['num_clients'],
    img_size=CONF['data']['img_size']
)

In [ ]:
# الخلية 3: تعريف النموذج (يدعم المسارين)
def create_model():
    arch = CONF['client']['model_arch']
    img_size = CONF['data']['img_size']

    if arch == "custom_cnn":
        # Path 1 Model
        model = keras.Sequential([
            keras.Input(shape=(img_size, img_size, 3)),
            layers.Conv2D(32, (3, 3), activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(128, activation="relu"),
            layers.Dense(1, activation="sigmoid"),
        ])
    elif arch == "mobilenet_v2":
        # Path 2 Model (Transfer Learning)
        base = tf.keras.applications.MobileNetV2(
            input_shape=(img_size, img_size, 3), include_top=False, weights="imagenet"
        )
        base.trainable = False # Freeze Base
        inputs = keras.Input(shape=(img_size, img_size, 3))
        x = base(inputs, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.2)(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

    # Optimizer
    opt_name = CONF['client']['optimizer']
    lr = CONF['client']['learning_rate']
    if opt_name == "sgd":
        opt = tf.keras.optimizers.SGD(learning_rate=lr, momentum=CONF['client']['momentum'])
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy", "AUC"])
    return model

In [ ]:
# الخلية 4: العميل والخادم والتشغيل
class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x, y):
        self.model = create_model()
        self.x, self.y = x, y
        self.weights = calculate_class_weights(y) # معالجة التوازن محلياً

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(
            self.x, self.y,
            epochs=CONF['client']['local_epochs'],
            batch_size=CONF['client']['batch_size'],
            class_weight=self.weights,
            verbose=0
        )
        return self.model.get_weights(), len(self.x), {}

def get_eval_fn():
    def evaluate(server_round, parameters, config):
        model = create_model()
        model.set_weights(parameters)
        loss, acc, auc = model.evaluate(test_ds, verbose=0)
        print(f"Round {server_round} | Loss: {loss:.4f} | Acc: {acc:.4f}")
        return loss, {"accuracy": acc, "auc": auc}
    return evaluate

# Strategy
strat_conf = CONF['server']
common = {
    "fraction_fit": 1.0, "min_fit_clients": 10, "min_available_clients": 10,
    "evaluate_fn": get_eval_fn()
}

if strat_conf['strategy'] == "fedavg":
    strategy = fl.server.strategy.FedAvg(**common)
elif strat_conf['strategy'] == "fedprox":
    strategy = fl.server.strategy.FedProx(proximal_mu=strat_conf['proximal_mu'], **common)
elif strat_conf['strategy'] == "fedadam":
    strategy = fl.server.strategy.FedAdam(eta=strat_conf['server_learning_rate'], eta_l=strat_conf['server_learning_rate'], tau=1e-9, **common)

# Run Simulation
def client_fn(cid):
    x, y = client_datasets[int(cid)]
    return FLClient(cid, x, y)

history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=CONF['data']['num_clients'],
    config=fl.server.ServerConfig(num_rounds=CONF['server']['num_rounds']),
    strategy=strategy,
    client_resources={"num_cpus": 2, "num_gpus": 0.2}
)

# --- SAVE RESULTS ---
# هذه الخطوة هي الأهم: ستحفظ كل شيء في مجلد باسم التجربة
save_experiment_results(history, client_datasets, CONF)